In [ ]:
from googleapiclient.http import MediaFileUpload
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

SERVICE_ACCOUNT_FILE = 'data/ark-cvat-points-key.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
sheets_service = build('sheets', 'v4', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

In [1]:
import xml.etree.ElementTree as ET

file_path = 'data/792_annotations/annotations.xml'

tree = ET.parse(file_path)
root = tree.getroot()

images_data = []
points, bboxes = [], []
for child in root:
    if child.tag == 'image':
        box = [subchild.attrib for subchild in child if subchild.tag == 'box']
        point = [subchild.attrib for subchild in child if subchild.tag == 'points']
        element_info = {
            'tag': child.tag,
            'attributes': child.attrib,
            'children': {'box': box, 'points': point}
        }
        images_data.append(element_info)

In [2]:
len(images_data)

461

In [3]:
images_data[1]

{'tag': 'image',
 'attributes': {'id': '1',
  'name': '-AeTIXz50g.jpg',
  'width': '1125',
  'height': '2000'},
 'children': {'box': [{'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '641.00',
    'ytl': '1766.00',
    'xbr': '764.00',
    'ybr': '1834.00',
    'z_order': '0'},
   {'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '376.00',
    'ytl': '1700.00',
    'xbr': '502.00',
    'ybr': '1759.00',
    'z_order': '0'},
   {'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '681.00',
    'ytl': '1984.00',
    'xbr': '805.00',
    'ybr': '2000.00',
    'z_order': '0'},
   {'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '595.00',
    'ytl': '1203.00',
    'xbr': '752.00',
    'ybr': '1252.00',
    'z_order': '0'},
   {'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '204.00',
    'ytl': '55.00',
    'xbr': '302.00',
    'ybr': '240.00',
    'z_order': '0'

In [25]:
# from PIL import Image, ImageDraw
# import os
# 
# label_properties = {
#                     'ignore': '#ababab',
#                     'МАК.ИЗДЕЛИЯ ЗНАТНЫЕ РОЖКИ ОСОБЫЕ №42 400Г': '#bdd5c3',
#                     'МАК.ИЗДЕЛИЯ ЗНАТНЫЕ СПАГЕТТИ 400Г': '#b6c1b6',
#                     'МАК.ИЗДЕЛИЯ ЗНАТНЫЕ СПИРАЛЬКИ №73 400Г': '#3825a6',
#                     'МАК.ИЗДЕЛИЯ PASTERONI ПИПЕ РИГАТЕ №126 400Г': '#0f9859',
#                     'МАК.ИЗДЕЛИЯ PASTERONI СПИРАЛЬКИ №125 400Г': '#9b9db8',
#                     'МАК.ИЗДЕЛИЯ PASTERONI РОЖКИ БОЛ. №130 400Г': '#a5a6bf',
#                     'МАК.ИЗДЕЛИЯ PASTERONI СПАГЕТТИ №114 450Г':	'#9a6b39',
#                     'МАК.ИЗДЕЛИЯ PASTERONI ТОРТИЛ. №127 400Г': '#0d94a8',
#                     'other': '#000001',
#                     '.other': '#0c004a',
#                     'PASTERONI Penne Rigate №129': '#33ddff',
#                     'PASTERONI Conchiglie Rigate №128': '#83e070',
#                     'PASTERONI Farfalle №170': '#fa32b7',
#                     'PASTERONI Tagliatelle Corte №132': '#34d1b7',
#                     'PASTERONI Cavatappi №187':	'#3df53d',
#                     'PASTERONI Riso №121': '#ff6a4d',
#                     'PASTERONI Grandine Ptitim №122': '#aaf0d1',
#                     'PASTERONI Lazagna': '#375263',
#                     'PASTERONI Vermicelli №136': '#83e070'
# }
# 
# def get_unique_filename(file_path):
#     base, extension = os.path.splitext(file_path)
#     counter = 1
#     while os.path.exists(file_path):
#         file_path = f"{base}_{counter}{extension}"
#         counter += 1
#     return file_path
# 
# def draw_point(img_name, bbox_orig, point_orig, point_color, destination):
#     image_path = img_name
#     original_image = Image.open(image_path)
#     new_point_coordinates = (point_orig[0] - bbox_orig[0],
#                              point_orig[1] - bbox_orig[1])
#     cropped_image = original_image.crop(bbox_orig)
#     draw = ImageDraw.Draw(cropped_image)
#     radius = 10
#     left_up_point = (new_point_coordinates[0] - radius, new_point_coordinates[1] - radius)
#     right_down_point = (new_point_coordinates[0] + radius, new_point_coordinates[1] + radius)
#     draw.ellipse([left_up_point, right_down_point], fill=point_color)
#     cropped_image.save(destination)
# 
# for image_data in images_data:
#     checked = 0
#     for point_data in image_data['children']['points']:
#         point_data_coord = list(map(int, map(float, point_data['points'].split(','))))
#         for box_data in image_data['children']['box']:
#             box_bbox = [int(float(box_data['xtl'])), int(float(box_data['ytl'])), 
#                         int(float(box_data['xbr'])), int(float(box_data['ybr']))]
#             if (box_bbox[0] < point_data_coord[0] < box_bbox[2] and 
#                     box_bbox[1] < point_data_coord[1] < box_bbox[3]):
#                 box_data['label'] = point_data['label']
#                 box_data['x'] = point_data_coord[0]
#                 box_data['y'] = point_data_coord[1]
#                 destination = get_unique_filename('data/cropped/' + 'cropped_' + image_data['attributes']['name'])
#                 draw_point('data/images/' + image_data['attributes']['name'], box_bbox, point_data_coord, label_properties[point_data['label']], destination)
#                 box_data['cropped_name'] = destination.split('/')[-1]
#                 checked = 1
#     image_data['checked'] = checked

In [6]:
from PIL import Image, ImageDraw
import os

def get_unique_filename(file_path):
    base, extension = os.path.splitext(file_path)
    counter = 1
    while os.path.exists(file_path):
        file_path = f"{base}_{counter}{extension}"
        counter += 1
    return file_path

def save_crop(img_name, bbox_orig, destination):
    image_path = img_name
    original_image = Image.open(image_path)
    cropped_image = original_image.crop(bbox_orig)
    cropped_image.save(destination)

In [8]:
for image_data in images_data:
    checked = 0
    for point_data in image_data['children']['points']:
        point_data_coord = list(map(int, map(float, point_data['points'].split(','))))
        for box_data in image_data['children']['box']:
            box_bbox = [int(float(box_data['xtl'])), int(float(box_data['ytl'])), 
                        int(float(box_data['xbr'])), int(float(box_data['ybr']))]
            if (box_bbox[0] < point_data_coord[0] < box_bbox[2] and 
                    box_bbox[1] < point_data_coord[1] < box_bbox[3]):
                box_data['label'] = point_data['label']
                box_data['x'] = point_data_coord[0]
                box_data['y'] = point_data_coord[1]
                destination = get_unique_filename('data/792_annotations/cropped/' + 'cropped_' + image_data['attributes']['name'])
                save_crop('data/792_annotations/images/' + image_data['attributes']['name'], box_bbox, destination)
                box_data['cropped_name'] = destination.split('/')[-1]
                checked = 1
    image_data['checked'] = checked

In [9]:
images_changed_data = [x for x in images_data if x['checked'] == 1]

In [10]:
len(images_changed_data)

121

In [11]:
images_changed_data[3]

{'tag': 'image',
 'attributes': {'id': '5',
  'name': '-_In80p42nZ1tA.jpg',
  'width': '736',
  'height': '1632'},
 'children': {'box': [{'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '502.00',
    'ytl': '1035.00',
    'xbr': '564.00',
    'ybr': '1063.00',
    'z_order': '0'},
   {'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '633.00',
    'ytl': '1054.00',
    'xbr': '694.00',
    'ybr': '1089.00',
    'z_order': '0'},
   {'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '569.00',
    'ytl': '1011.00',
    'xbr': '629.00',
    'ybr': '1053.00',
    'z_order': '0'},
   {'label': 'other',
    'occluded': '0',
    'source': 'manual',
    'xtl': '204.00',
    'ytl': '1158.00',
    'xbr': '269.00',
    'ybr': '1271.00',
    'z_order': '0',
    'x': 239,
    'y': 1221,
    'cropped_name': 'cropped_-_In80p42nZ1tA_5.jpg'},
   {'label': 'ignore',
    'occluded': '0',
    'source': 'manual',
    'xtl': '207.00

In [12]:
for image_data in images_changed_data:
    image_data['children']['box'] = [x for x in image_data['children']['box'] if x['label'] != 'ignore']
    try:
        del image_data['children']['points']
    except:
        pass

In [13]:
images_changed_data[3]

{'tag': 'image',
 'attributes': {'id': '5',
  'name': '-_In80p42nZ1tA.jpg',
  'width': '736',
  'height': '1632'},
 'children': {'box': [{'label': 'other',
    'occluded': '0',
    'source': 'manual',
    'xtl': '204.00',
    'ytl': '1158.00',
    'xbr': '269.00',
    'ybr': '1271.00',
    'z_order': '0',
    'x': 239,
    'y': 1221,
    'cropped_name': 'cropped_-_In80p42nZ1tA_5.jpg'},
   {'label': 'other',
    'occluded': '0',
    'source': 'manual',
    'xtl': '171.00',
    'ytl': '223.00',
    'xbr': '221.00',
    'ybr': '324.00',
    'z_order': '0',
    'x': 194,
    'y': 275,
    'cropped_name': 'cropped_-_In80p42nZ1tA.jpg'},
   {'label': 'other',
    'occluded': '0',
    'source': 'manual',
    'xtl': '211.00',
    'ytl': '1315.00',
    'xbr': '270.00',
    'ybr': '1422.00',
    'z_order': '0',
    'x': 238,
    'y': 1380,
    'cropped_name': 'cropped_-_In80p42nZ1tA_8.jpg'},
   {'label': 'Myaso в банке Оленина тушеная в/с 338 гр.',
    'occluded': '0',
    'source': 'manual',
   

In [12]:
def upload_image_to_drive(drive_service, file_path, file_name):
    file_metadata = {
        'name': file_name,
        'mimeType': 'image/jpeg'
    }
    media = MediaFileUpload(file_path, mimetype='image/jpeg')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

def get_image_url(drive_service, file_id):
    user_permission = {
        'type': 'anyone',
        'role': 'writer'
    }
    drive_service.permissions().create(fileId=file_id, body=user_permission).execute()
    
    return f"https://drive.google.com/uc?id={file_id}"

In [11]:
# result = []
# max_points = 0
# for image_data in images_changed_data:
#     column = [None, None, None, image_data['attributes']['id'], image_data['attributes']['name'], None]
#     cnt = 0
#     for changed_point in image_data['children']['box']:
#         if changed_point['label'] != 'ignore':
#             file_id = upload_image_to_drive(drive_service, f"data/cropped/{changed_point['cropped_name']}", changed_point['cropped_name'])
#             image_url = get_image_url(drive_service, file_id)
#             column.append(image_url)
#             column[0] = changed_point['label']
#             cnt += 1
#     column[5] = cnt 
#     if max_points < cnt:
#         max_points = cnt
# 
#     result.append(column)
# 
# result.sort(key=lambda x: x[0])
# result.insert(0, ['Название',	'фото эталон',	'task id',	'id фото в task',	'название фото',	'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)])

In [12]:
# spreadsheet_body = {'properties': {'title': 'test2'}}
# spreadsheet = sheets_service.spreadsheets().create(body=spreadsheet_body, fields='spreadsheetId').execute()
# SPREADSHEET_ID = spreadsheet.get('spreadsheetId')
# 
# values = result
# range_ = 'Sheet1!A1'
# value_input_option = 'USER_ENTERED'
# request = sheets_service.spreadsheets().values().update(
#     spreadsheetId=SPREADSHEET_ID, range=range_, valueInputOption=value_input_option, body={'values': values})
# response = request.execute()
# 
# user_permission = {
#     'type': 'user',
#     'role': 'writer',
#     'emailAddress': 'abilay5787@gmail.com'
# }
# drive_service.permissions().create(fileId=SPREADSHEET_ID, body=user_permission).execute()

In [15]:
def column_index_to_name(index):
    name = ""
    while index > 0:
        index, remainder = divmod(index - 1, 26)
        name = chr(65 + remainder) + name
    return name

result = []
max_points = 0

for image_data in images_changed_data:
    column = [None, None, None, image_data['attributes']['id'], image_data['attributes']['name'], None] # + [None] * max_points
    cnt = 0

    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            # file_id = upload_image_to_drive(drive_service, f"data/cropped/{changed_point['cropped_name']}", changed_point['cropped_name'])
            # image_url = get_image_url(drive_service, file_id)
            # 
            # column.append(f'=IMAGE("{image_url}", 4, 150, 150)')
            column[0] = changed_point['label']
            cnt += 1

    column[5] = cnt 
    if max_points < cnt:
        max_points = cnt

    result.append(column)
    
result.sort(key=lambda x: x[0])

In [ ]:
headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
result.insert(0, headers)

In [26]:
spreadsheet_body = {'properties': {'title': 'test2'}}
spreadsheet = sheets_service.spreadsheets().create(body=spreadsheet_body, fields='spreadsheetId').execute()
SPREADSHEET_ID = spreadsheet.get('spreadsheetId')

values = result
fixed_columns = 6
total_columns = fixed_columns + max_points
end_column_name = column_index_to_name(total_columns)
range_ = f'Sheet1!A1:{end_column_name}'
value_input_option = 'USER_ENTERED'
request = sheets_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, range=range_, valueInputOption=value_input_option, body={'values': values})
response = request.execute()

user_permission = {
    'type': 'user',
    'role': 'writer',
    'emailAddress': 'abilay5787@gmail.com'
}
drive_service.permissions().create(fileId=SPREADSHEET_ID, body=user_permission).execute()

image_width = 150
image_height = 150

sheet_id = 0
batch_update_spreadsheet_request_body = {
    "requests": [
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "COLUMNS",
                    "startIndex": 6,
                    "endIndex": 6 + max_points
                },
                "properties": {
                    "pixelSize": image_width
                },
                "fields": "pixelSize"
            }
        },
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "ROWS",
                    "startIndex": 1,
                    "endIndex": len(values)
                },
                "properties": {
                    "pixelSize": image_height
                },
                "fields": "pixelSize"
            }
        }
    ]
}

sheets_service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body=batch_update_spreadsheet_request_body).execute()

{'kind': 'drive#permission',
 'id': '02333895286842277568',
 'type': 'user',
 'role': 'writer'}

In [23]:
SPREADSHEET_ID

'1UbKpZITAUrnNZqHMH_NlRBqEwR256YNKAEpIZEHEvQE'

In [33]:
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from PIL import Image as PILImage
import os

def column_index_to_name(index):
    name = ""
    while index > 0:
        index, remainder = divmod(index - 1, 26)
        name = chr(65 + remainder) + name
    return name

fixed_img_size = (200, 200)

wb = Workbook()
ws = wb.active

max_points = 0
for image_data in images_changed_data:
    point_count = sum(1 for cp in image_data['children']['box'] if cp['label'] != 'ignore')
    max_points = max(max_points, point_count)

headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
ws.append(headers)

for image_data in images_changed_data:
    name = next((item['label'] for item in image_data['children']['box'] if item['label'] != 'ignore'), None)
    row_data = [name, None, None, image_data['attributes']['id'], image_data['attributes']['name']]
    cnt = 0

    img_row = ws.max_row + 1

    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            image_path = f"data/cropped/{changed_point['cropped_name']}"
            if os.path.exists(image_path):
                cnt += 1
                img = PILImage.open(image_path)
                img = img.resize(fixed_img_size, PILImage.Resampling.LANCZOS)
                img_path_resized = f"data/resized/resized_{changed_point['cropped_name']}"
                img.save(img_path_resized)
                img_column = column_index_to_name(6 + cnt)
                ws.add_image(Image(img_path_resized), f'{img_column}{img_row}')

                actual_height = img.size[1]
                ws.row_dimensions[img_row].height = actual_height * 0.75

    row_data.append(cnt)
    row_data.extend([None] * (max_points - cnt))
    ws.append(row_data)

for i in range(6, 6 + max_points):
    ws.column_dimensions[column_index_to_name(i)].width = fixed_img_size[0] / 7

excel_file = 'data/data_with_images_corrected.xlsx'
wb.save(excel_file)

In [41]:
import pandas as pd
import pygsheets
def upload_excel_to_drive(drive_service, file_path, file_name):
    file_metadata = {
        'name': file_name,
        'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    }
    media = MediaFileUpload(file_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

def import_excel_to_google_sheets(excel_file_path, google_sheets_id, google_sheets_sheet_name):
    # Авторизация в Google Sheets
    gc = pygsheets.authorize(service_account_file=SERVICE_ACCOUNT_FILE)

    # Открываем таблицу по ID
    spreadsheet = gc.open_by_key(google_sheets_id)

    # Указываем лист, на котором хотим импортировать данные
    worksheet = spreadsheet.worksheet_by_title(google_sheets_sheet_name)

    # Считываем данные из Excel-файла
    df = pd.read_excel(excel_file_path)

    # Загружаем данные в Google Sheets
    worksheet.set_dataframe(df, start='A1')  # Загрузить данные начиная с ячейки A1

    # Сохраняем изменения
    spreadsheet.drive_client.upload_file(excel_file_path)

# Путь к вашему Excel-файлу
excel_file_path = 'data/data_with_images_corrected.xlsx'

# Название листа, на котором вы хотите импортировать данные
google_sheets_sheet_name = 'Sheet1'

# Загрузка файла на Google Диск
excel_file_name = 'data_with_images_corrected.xlsx'
file_metadata = {
    'name': excel_file_name,
    'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
}
media = MediaFileUpload(excel_file_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
excel_file_id = file.get('id')

# Вызываем функцию для импорта данных из Excel-файла в Google Sheets
import_excel_to_google_sheets(excel_file_path, excel_file_id, google_sheets_sheet_name)

user_permission = {
    'type': 'user',
    'role': 'writer',
    'emailAddress': 'abilay5787@gmail.com'
}
drive_service.permissions().create(fileId=SPREADSHEET_ID, body=user_permission).execute()

# Формат, в котором будет загружен файл Excel в Google Sheets
sheet_id = 0  # Если у вас только один лист
formula = f'=IMPORTRANGE("https://drive.google.com/uc?id={excel_file_id}", "Sheet1!A1")'
value_input_option = 'RAW'
body = {
    'requests': [{
        'pasteData': {
            'data': formula,
            'type': 'PASTE_FORMULA',
            'delimiter': ',',
            'coordinate': {
                'sheetId': sheet_id,
                'rowIndex': 0,
                'columnIndex': 0
            }
        }
    }]
}

# Выполнение запроса на импорт Excel-файла в Google Sheets
sheets_service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body=body
).execute()

HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1SHuGmW1Zmprt2WO3abIhqH6FUY5iZ6H9?includeGridData=false&fields=%2A&alt=json returned "This operation is not supported for this document". Details: "This operation is not supported for this document">

In [10]:
def column_index_to_name(index):
    name = ""
    while index > 0:
        index, remainder = divmod(index - 1, 26)
        name = chr(65 + remainder) + name
    return name

result = []
max_points = 0
image_width = 200
image_height = 200

for image_data in images_changed_data[:5]:
    column = [None, None, None, image_data['attributes']['id'], image_data['attributes']['name'], None] # + [None] * max_points
    cnt = 0

    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            file_id = upload_image_to_drive(drive_service, f"data/cropped/{changed_point['cropped_name']}", changed_point['cropped_name'])
            image_url = get_image_url(drive_service, file_id)
            
            column.append(f'=IMAGE("{image_url}", 4, {image_width}, {image_height})')
            column[0] = changed_point['label']
            cnt += 1

    column[5] = cnt 
    if max_points < cnt:
        max_points = cnt

    result.append(column)

result.sort(key=lambda x: x[0])

headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
result.insert(0, headers)

spreadsheet_body = {'properties': {'title': 'test'}}
spreadsheet = sheets_service.spreadsheets().create(body=spreadsheet_body, fields='spreadsheetId').execute()
SPREADSHEET_ID = spreadsheet.get('spreadsheetId')

values = result
fixed_columns = 6
total_columns = fixed_columns + max_points
end_column_name = column_index_to_name(total_columns)
range_ = f'Sheet1!A1:{end_column_name}'
value_input_option = 'USER_ENTERED'
request = sheets_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, range=range_, valueInputOption=value_input_option, body={'values': values})
response = request.execute()

user_permission = {
    'type': 'user',
    'role': 'writer',
    'emailAddress': 'abilay5787@gmail.com'
}
drive_service.permissions().create(fileId=SPREADSHEET_ID, body=user_permission).execute()

sheet_id = 0
batch_update_spreadsheet_request_body = {
    "requests": [
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "COLUMNS",
                    "startIndex": 6,
                    "endIndex": 6 + max_points
                },
                "properties": {
                    "pixelSize": image_width
                },
                "fields": "pixelSize"
            }
        },
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "ROWS",
                    "startIndex": 1,
                    "endIndex": len(values)
                },
                "properties": {
                    "pixelSize": image_height
                },
                "fields": "pixelSize"
            }
        }
    ]
}

sheets_service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body=batch_update_spreadsheet_request_body).execute()

{'spreadsheetId': '1Q6PZHPyvxvbcCW8ybw3wnMOZOyJEcOzOCjItn1lWNpo',
 'replies': [{}, {}]}

In [13]:
images_changed_data[0]

{'tag': 'image',
 'attributes': {'id': '1',
  'name': '-76yXaMQHw.jpg',
  'width': '1125',
  'height': '2000'},
 'children': {'box': [{'label': 'МАК.ИЗДЕЛИЯ PASTERONI СПИРАЛЬКИ №125 400Г',
    'occluded': '0',
    'source': 'manual',
    'xtl': '253.00',
    'ytl': '1306.00',
    'xbr': '563.00',
    'ybr': '1773.00',
    'z_order': '0',
    'x': 425,
    'y': 1509,
    'cropped_name': 'cropped_-76yXaMQHw_1.jpg'},
   {'label': 'МАК.ИЗДЕЛИЯ PASTERONI ТОРТИЛ. №127 400Г',
    'occluded': '0',
    'source': 'manual',
    'xtl': '499.00',
    'ytl': '633.00',
    'xbr': '816.00',
    'ybr': '1159.00',
    'z_order': '0',
    'x': 650,
    'y': 869,
    'cropped_name': 'cropped_-76yXaMQHw_2.jpg'},
   {'label': 'other',
    'occluded': '0',
    'source': 'manual',
    'xtl': '213.00',
    'ytl': '730.00',
    'xbr': '498.00',
    'ybr': '1097.00',
    'z_order': '0',
    'x': 332,
    'y': 851,
    'cropped_name': 'cropped_-76yXaMQHw_3.jpg'},
   {'label': 'МАК.ИЗДЕЛИЯ PASTERONI СПИРАЛЬКИ №125

In [16]:
from copy import deepcopy

def column_index_to_name(index):
    name = ""
    while index > 0:
        index, remainder = divmod(index - 1, 26)
        name = chr(65 + remainder) + name
    return name

result = []
max_points = 0
image_width = 200
image_height = 200
copied_data = deepcopy(images_changed_data)

for image_data in copied_data[:5]:
    label_groups = {}
    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            label = changed_point['label']
            if label not in label_groups:
                label_groups[label] = []
            label_groups[label].append(changed_point)

    for label, points in label_groups.items():
        image_urls = []
        for point in points:
            file_id = upload_image_to_drive(drive_service, f"data/cropped/{point['cropped_name']}", point['cropped_name'])
            image_url = get_image_url(drive_service, file_id)
            image_urls.append(f'=IMAGE("{image_url}", 4, {image_width}, {image_height})')

        column = [label, None, None, image_data['attributes']['id'], image_data['attributes']['name'], len(points)] + image_urls

        if len(points) > max_points:
            max_points = len(points)

        result.append(column)

result.sort(key=lambda x: x[0])


headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
result.insert(0, headers)

spreadsheet_body = {'properties': {'title': 'test'}}
spreadsheet = sheets_service.spreadsheets().create(body=spreadsheet_body, fields='spreadsheetId').execute()
SPREADSHEET_ID = spreadsheet.get('spreadsheetId')

values = result
fixed_columns = 6
total_columns = fixed_columns + max_points
end_column_name = column_index_to_name(total_columns)
range_ = f'Sheet1!A1:{end_column_name}'
value_input_option = 'USER_ENTERED'
request = sheets_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, range=range_, valueInputOption=value_input_option, body={'values': values})
response = request.execute()

user_permission = {
    'type': 'user',
    'role': 'writer',
    'emailAddress': 'abilay5787@gmail.com'
}
drive_service.permissions().create(fileId=SPREADSHEET_ID, body=user_permission).execute()

sheet_id = 0
batch_update_spreadsheet_request_body = {
    "requests": [
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "COLUMNS",
                    "startIndex": 6,
                    "endIndex": 6 + max_points
                },
                "properties": {
                    "pixelSize": image_width
                },
                "fields": "pixelSize"
            }
        },
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "ROWS",
                    "startIndex": 1,
                    "endIndex": len(values)
                },
                "properties": {
                    "pixelSize": image_height
                },
                "fields": "pixelSize"
            }
        }
    ]
}

sheets_service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body=batch_update_spreadsheet_request_body).execute()

{'spreadsheetId': '1J0vcRmcZ5__43-66_RlrZEPlQ9BYJQuPIfN4v3oYPTU',
 'replies': [{}, {}]}

In [35]:
from copy import deepcopy

def column_index_to_name(index):
    name = ""
    while index > 0:
        index, remainder = divmod(index - 1, 26)
        name = chr(65 + remainder) + name
    return name

result = []
max_points = 0
image_width = 200
image_height = 200
copied_data = deepcopy(images_changed_data)

for image_data in copied_data[:5]:
    label_groups = {}
    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            label = changed_point['label']
            if label not in label_groups:
                label_groups[label] = []
            label_groups[label].append(changed_point)

    for label, points in label_groups.items():
        image_urls = []
        for point in points:
            file_id = upload_image_to_drive(drive_service, f"data/cropped/{point['cropped_name']}", point['cropped_name'])
            image_url = get_image_url(drive_service, file_id)
            image_urls.append(f'=IMAGE("{image_url}", 4, {image_width}, {image_height})')

        column = [label, None, None, image_data['attributes']['id'], image_data['attributes']['name'], len(points)] + image_urls

        if len(points) > max_points:
            max_points = len(points)

        result.append(column)

result.sort(key=lambda x: x[0])


headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
result.insert(0, headers)

In [42]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter
from copy import deepcopy

wb = openpyxl.Workbook()
ws = wb.active

result = []
max_points = 0
copied_data = deepcopy(images_changed_data)

for image_data in copied_data:
    label_groups = {}
    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            label = changed_point['label']
            if label not in label_groups:
                label_groups[label] = []
            label_groups[label].append(changed_point)

    for label, points in label_groups.items():
        image_file_paths = []
        for point in points:
            local_file_path = f"data/cropped/{point['cropped_name']}"
            image_file_paths.append(local_file_path)

        column = [label, None, None, image_data['attributes']['id'], image_data['attributes']['name'], len(points)] + image_file_paths

        if len(points) > max_points:
            max_points = len(points)

        result.append(column)

result.sort(key=lambda x: x[0])

headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
ws.append(headers)

fixed_img_width_px = 200

def pixels_to_width_units(pixels):
    return pixels / 7

for row, data in enumerate(result, start=2):
    for col, cell_value in enumerate(data, start=1):
        if not (isinstance(cell_value, str) and cell_value.startswith("data/cropped/")):
            ws.cell(row=row, column=col, value=cell_value)

for row, data in enumerate(result, start=2):
    max_height_in_row = 0
    for col, cell_value in enumerate(data, start=1):
        if isinstance(cell_value, str) and cell_value.startswith("data/cropped/"):
            img = Image(cell_value)
            img.width, img.height = fixed_img_width_px, img.height
            max_height_in_row = max(max_height_in_row, img.height)
            ws.add_image(img, get_column_letter(col) + str(row))

    if max_height_in_row > 0:
        ws.row_dimensions[row].height = max_height_in_row * 0.75

ws.column_dimensions['A'].width = 40
ws.column_dimensions['E'].width = 20

for i in range(6, 6 + max_points):
    ws.column_dimensions[get_column_letter(i)].width = pixels_to_width_units(fixed_img_width_px)

wb.save('data/output.xlsx')

In [13]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter
from copy import deepcopy
from PIL import Image as PILImage

def pixels_to_width_units(pixels):
    return pixels / 7

def get_image_size_with_aspect_ratio(image_path, max_width):
    with PILImage.open(image_path) as img:
        original_width, original_height = img.size
        if original_width > max_width:
            scaling_factor = max_width / original_width
            new_width = max_width
            new_height = original_height * scaling_factor
            return new_width, new_height
        else:
            return original_width, original_height

# Initialize workbook and worksheet
wb = openpyxl.Workbook()
ws = wb.active

# Data processing
result = []
max_points = 0
copied_data = deepcopy(images_changed_data)

for image_data in copied_data:
    label_groups = {}
    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            label = changed_point['label']
            if label not in label_groups:
                label_groups[label] = []
            label_groups[label].append(changed_point)

    for label, points in label_groups.items():
        image_file_paths = []
        for point in points:
            local_file_path = f"data/878_annotations/cropped/{point['cropped_name']}"
            image_file_paths.append(local_file_path)

        column = [label, None, None, image_data['attributes']['id'], image_data['attributes']['name'], len(points)] + image_file_paths

        if len(points) > max_points:
            max_points = len(points)

        result.append(column)

result.sort(key=lambda x: x[0])

headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
ws.append(headers)

max_image_widths = [0] * max_points

for row, data in enumerate(result, start=2):
    max_height_in_row = 0
    for col, cell_value in enumerate(data, start=1):
        if isinstance(cell_value, str) and cell_value.startswith("data/878_annotations/cropped/"):
            img_width, img_height = get_image_size_with_aspect_ratio(cell_value, 200) 
            img = Image(cell_value)
            img.width, img.height = img_width, img_height
            max_height_in_row = max(max_height_in_row, img.height)
            ws.add_image(img, get_column_letter(col) + str(row))

            image_col_index = col - 6
            if 0 <= image_col_index < len(max_image_widths):
                max_image_widths[image_col_index] = max(max_image_widths[image_col_index], img_width)
        else:
            ws.cell(row=row, column=col, value=cell_value)

    if max_height_in_row > 0:
        ws.row_dimensions[row].height = max_height_in_row * 0.75

for i, max_width in enumerate(max_image_widths, start=6):
    if max_width > 0:
        ws.column_dimensions[get_column_letter(i)].width = pixels_to_width_units(max_width)

ws.column_dimensions['A'].width = 40
ws.column_dimensions['E'].width = 20

wb.save('data/878_annotations/output2.xlsx')

In [ ]:
def column_index_to_name(index):
    name = ""
    while index > 0:
        index, remainder = divmod(index - 1, 26)
        name = chr(65 + remainder) + name
    return name

result = []
max_points = 0
image_width = 200
image_height = 200

for image_data in images_changed_data[:5]:
    column = [None, None, None, image_data['attributes']['id'], image_data['attributes']['name'], None] # + [None] * max_points
    cnt = 0

    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            file_id = upload_image_to_drive(drive_service, f"data/cropped/{changed_point['cropped_name']}", changed_point['cropped_name'])
            image_url = get_image_url(drive_service, file_id)
            
            column.append(f'=IMAGE("{image_url}", 4, {image_width}, {image_height})')
            column[0] = changed_point['label']
            cnt += 1

    column[5] = cnt 
    if max_points < cnt:
        max_points = cnt

    result.append(column)

result.sort(key=lambda x: x[0])

headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
result.insert(0, headers)

spreadsheet_body = {'properties': {'title': 'test'}}
spreadsheet = sheets_service.spreadsheets().create(body=spreadsheet_body, fields='spreadsheetId').execute()
SPREADSHEET_ID = spreadsheet.get('spreadsheetId')

values = result
fixed_columns = 6
total_columns = fixed_columns + max_points
end_column_name = column_index_to_name(total_columns)
range_ = f'Sheet1!A1:{end_column_name}'
value_input_option = 'USER_ENTERED'
request = sheets_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, range=range_, valueInputOption=value_input_option, body={'values': values})
response = request.execute()

user_permission = {
    'type': 'user',
    'role': 'writer',
    'emailAddress': 'abilay5787@gmail.com'
}
drive_service.permissions().create(fileId=SPREADSHEET_ID, body=user_permission).execute()

sheet_id = 0
batch_update_spreadsheet_request_body = {
    "requests": [
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "COLUMNS",
                    "startIndex": 6,
                    "endIndex": 6 + max_points
                },
                "properties": {
                    "pixelSize": image_width
                },
                "fields": "pixelSize"
            }
        },
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "ROWS",
                    "startIndex": 1,
                    "endIndex": len(values)
                },
                "properties": {
                    "pixelSize": image_height
                },
                "fields": "pixelSize"
            }
        }
    ]
}

sheets_service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body=batch_update_spreadsheet_request_body).execute()

In [ ]:
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from PIL import Image as PILImage
import os

def column_index_to_name(index):
    name = ""
    while index > 0:
        index, remainder = divmod(index - 1, 26)
        name = chr(65 + remainder) + name
    return name

fixed_img_size = (200, 200)

wb = Workbook()
ws = wb.active

max_points = 0
for image_data in images_changed_data:
    point_count = sum(1 for cp in image_data['children']['box'] if cp['label'] != 'ignore')
    max_points = max(max_points, point_count)

headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
ws.append(headers)

for image_data in images_changed_data:
    name = next((item['label'] for item in image_data['children']['box'] if item['label'] != 'ignore'), None)
    row_data = [name, None, None, image_data['attributes']['id'], image_data['attributes']['name']]
    cnt = 0

    img_row = ws.max_row + 1

    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            image_path = f"data/cropped/{changed_point['cropped_name']}"
            if os.path.exists(image_path):
                cnt += 1
                img = PILImage.open(image_path)
                img = img.resize(fixed_img_size, PILImage.Resampling.LANCZOS)
                img_path_resized = f"data/resized/resized_{changed_point['cropped_name']}"
                img.save(img_path_resized)
                img_column = column_index_to_name(6 + cnt)
                ws.add_image(Image(img_path_resized), f'{img_column}{img_row}')

                actual_height = img.size[1]
                ws.row_dimensions[img_row].height = actual_height * 0.75

    row_data.append(cnt)
    row_data.extend([None] * (max_points - cnt))
    ws.append(row_data)

for i in range(6, 6 + max_points):
    ws.column_dimensions[column_index_to_name(i)].width = fixed_img_size[0] / 7

excel_file = 'data/data_with_images_corrected.xlsx'
wb.save(excel_file)

In [16]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter
from copy import deepcopy
from PIL import Image as PILImage

def pixels_to_width_units(pixels):
    return pixels / 7

def get_image_size_with_aspect_ratio(image_path, max_width):
    with PILImage.open(image_path) as img:
        original_width, original_height = img.size
        if original_width > max_width:
            scaling_factor = max_width / original_width
            new_width = max_width
            new_height = original_height * scaling_factor
            return new_width, new_height
        else:
            return original_width, original_height

wb = openpyxl.Workbook()
ws = wb.active

result = []
max_points = 0
copied_data = deepcopy(images_changed_data)

for image_data in copied_data:
    label_groups = {}
    for changed_point in image_data['children']['box']:
        if changed_point['label'] != 'ignore':
            label = changed_point['label']
            if label not in label_groups:
                label_groups[label] = []
            label_groups[label].append(changed_point)

    for label, points in label_groups.items():
        image_file_paths = []
        for point in points:
            local_file_path = f"data/878_annotations/cropped/{point['cropped_name']}"
            image_file_paths.append(local_file_path)

        column = [label, None, None, image_data['attributes']['id'], image_data['attributes']['name'], len(points)] + image_file_paths

        if len(points) > max_points:
            max_points = len(points)

        result.append(column)

result.sort(key=lambda x: x[0])

headers = ['Название', 'фото эталон', 'task id', 'id фото в task', 'название фото', 'количество точек на фото'] + [f'фото{x}' for x in range(1, max_points + 1)]
ws.append(headers)

max_image_widths = [0] * max_points

for row, data in enumerate(result, start=2):
    max_height_in_row = 0
    for col, cell_value in enumerate(data, start=1):
        if isinstance(cell_value, str) and cell_value.startswith("data/878_annotations/cropped/"):
            img_width, img_height = get_image_size_with_aspect_ratio(cell_value, 200) 
            img = Image(cell_value)
            img.width, img.height = img_width, img_height
            max_height_in_row = max(max_height_in_row, img.height)
            ws.add_image(img, get_column_letter(col) + str(row))

            image_col_index = col - 6
            if 0 <= image_col_index < len(max_image_widths):
                max_image_widths[image_col_index] = max(max_image_widths[image_col_index], img_width)
        else:
            ws.cell(row=row, column=col, value=cell_value)

    if max_height_in_row > 0:
        ws.row_dimensions[row].height = max_height_in_row * 0.75

for i, max_width in enumerate(max_image_widths, start=6):
    if max_width > 0:
        ws.column_dimensions[get_column_letter(i)].width = pixels_to_width_units(max_width)

ws.column_dimensions['A'].width = 40
ws.column_dimensions['E'].width = 20

wb.save('data/878_annotations/output2.xlsx')